In [48]:
from __future__ import unicode_literals
import csv
import re
import numpy as np
import utility as ut

import features_final as f

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from xgboost import XGBClassifier

import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from itertools import compress

In [49]:
def clean(filename,filenameWrite):
    
    cleaned_data = []
    with open(filename,'r') as file:
        reader = csv.reader(file)
        for row in reader:
            #print len(row)
            title = row[2]
            description = row[6]
            cleanedDescription = ut.cleanDescription(description)
            cleanedTitle = ut.cleanText(title, False)
            
            # normalize price
            price = float(row[7])
            country = row[0]
            newPrice = ut.normalizePrice(price, country)
            
            cleanedRow = [row[0],row[1],cleanedTitle,row[3],row[4],row[5],cleanedDescription, newPrice,row[8]]
            cleaned_data.append(cleanedRow)

    writer = csv.writer(open(filenameWrite, 'w'))
    writer.writerows(cleaned_data)

In [3]:
w2v_model = f.loadWordVectors('google_news_word_vectors/GoogleNews-vectors-negative300.bin')

Loading Word Vectors............
Loaded Word Vectors.............


In [28]:
#Should be done only once

print("Cleaning the text in the train data............")
clean('Data/training/data_train.csv','Data/training/data_train_clean.csv')

Cleaning the text in the train data............


In [50]:
#Pass the features that you want to test. The order of features are as given below
#word2VecFlag 
#lengthAndBooleanFlag
#tfidfLabelIntersectFlag
#numericFlag
#denseEntropySKUFlag

print("Computing Features............")
X = f.extract_features("Data/training/data_train_clean.csv",w2v_model,False,True,False,False,False)
X_input = X.astype(np.float)

print("Features Computed............")

y_input = np.loadtxt("data/training/clarity_train.labels", dtype=float)


Computing Features............
Features Computed............


In [51]:
#Normalize the Feature Inputs

X_std = (X_input - X_input.min(axis=0)) / (X_input.max(axis=0) - X_input.min(axis=0))
X_input = X_std
X_input = X_input[:,~np.all(np.isnan(X_input), axis=0)]


In [24]:
#Pickle the files, load the pickled files
#Use this only if you want to pickle the features and use it later, else don't run this cell

import pickle

#pickle.dump(X_input, open( "X_input_numerical.p", "wb" ))
#pickle.dump(y_input, open( "y_input_numerical.p", "wb" ))

#Load either the X_input_binary or X_input_numerical depending on the features to be tested
#loaded_X_input = pickle.load( open( "X_input_binary.p", "rb" ) )
#loaded_y_input = pickle.load( open( "y_input_binary.p", "rb" ) )

#y_input = np.loadtxt("data/training/clarity_train.labels", dtype=float)

X_input = loaded_X_input

In [52]:
from itertools import compress
y_new = list(y_input==1)
#result = list(compress(range(len(y_new),y_new)))
result_pos = [i for i,x in enumerate(y_new) if x]
result_neg = [i for i,x in enumerate(y_new) if not x]

X_input_pos = X_input[result_pos]
y_input_pos = np.ones(len(X_input_pos));

X_input_neg = X_input[result_neg]
y_input_neg = np.zeros(len(X_input_neg));


In [ ]:
kFoldResults = ut.splitKFold(X_input,y_input,5)

F1 = []
Acc = []
Precision = []
Roc = []
Recall = []
Avg_Prec = []


for resultGenerator in kFoldResults:
    trainIndices = resultGenerator[0]
    testIndices = resultGenerator[1]
    X_train = list()
    y_train=list()
    X_test=list()
    y_test=list()
    for trainRow in trainIndices:
        X_train.append(X_input[trainRow])
        y_train.append(y_input[trainRow])

    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
        
    print("Features Computed for Testing data............")
        
    for testRow in testIndices:
        X_test.append(X_input[testRow])
        y_test.append(y_input[testRow])

    y_bools = list(y_train==1)
    train_split_pos = [i for i,x in enumerate(y_bools) if x]
    train_split_neg = [i for i,x in enumerate(y_bools) if not x]

    X_train_input_pos = X_input[train_split_pos]
    y_train_input_pos = np.ones(len(train_split_pos));

    X_train_input_neg = X_input[train_split_neg]
    y_train_input_neg = np.zeros(len(train_split_neg));
    
    print(X_train.shape)
    print(X_train_input_pos.shape)
    print(X_train_input_neg.shape)
    
    kFoldTrainingSplits = 6
    kFoldTrainingData = ut.splitKFold(X_train_input_pos,y_train_input_pos,kFoldTrainingSplits)

    models = list()
    
    for resultGenerator in kFoldTrainingData:
        bigIndices = resultGenerator[0]
        smallIndices = resultGenerator[1]
        
        X_train_input_small = list()
        y_train_input_small = list()
        for smallIndex in smallIndices:
            X_train_input_small.append(X_input[smallIndex])
            y_train_input_small.append(y_input[smallIndex])

        X_train_input_small = np.asarray(X_train_input_small)
        y_train_input_small = np.asarray(y_train_input_small)

        X_train_new = np.concatenate((X_train_input_small,X_train_input_neg),axis=0);
        y_train_new = np.concatenate((y_train_input_small,y_train_input_neg),axis=0);

        p = np.random.permutation(len(X_train_new))
        X_train_new = X_train_new[p]
        y_train_new = y_train_new[p]
    

        # Model training
        print("Training the Model............")

        #Replace with appropriate model, LogisticRegression, RandomForestClassifier for testing     
        model = LogisticRegression()
        
        models.append(model)
        models[len(models)-1].fit(X_train_new,y_train_new)

        
    print("Preciting labels from all classes............")
    y_predictions = list()
    for model in models:
        y_pred = model.predict(X_test)

        y_predictions.append(y_pred)

    print("Finalzing test data classes............")
    y_predictions_sum = map(sum,zip(*y_predictions))
    y_predictions_sum = np.asarray(y_predictions_sum)

    y_predictions_bool = (y_predictions_sum >= (kFoldTrainingSplits/2))
    y_predictions_bool = y_predictions_bool.astype(int)
    y_predictions_classes = list((y_predictions_bool))
    
    
    # Calculate the performance measures
    acc = accuracy_score(y_test,y_predictions_classes)
    f1 = f1_score(y_test,y_predictions_classes)
    roc = roc_auc_score(y_test,y_predictions_classes)
    precision = precision_score(y_test, y_predictions_classes)
    recall = recall_score(y_test,y_predictions_classes)
    avg_prec = average_precision_score(y_test, y_predictions_classes)

    Acc.append(acc)
    F1.append(f1)
    Roc.append(roc)
    Precision.append(precision)
    Recall.append(recall)
    Avg_Prec.append(avg_prec)

    print acc
    print f1 
    print roc
    print precision
    print recall
    print avg_prec 

In [ ]:
# Get the average values
print np.mean(Acc)
print np.mean(Roc)
print np.mean(Precision)
print np.mean(F1)
print np.mean(Recall)
print np.mean(avg_prec)